In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3d-mnist/voxelgrid.py
/kaggle/input/3d-mnist/train_point_clouds.h5
/kaggle/input/3d-mnist/test_point_clouds.h5
/kaggle/input/3d-mnist/plot3D.py
/kaggle/input/3d-mnist/full_dataset_vectors.h5


In [47]:
import torch
import torchvision.models

In [121]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        
        # Layer Definition
        vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)
        self.vgg = torch.nn.Sequential(*list(vgg16_bn.features.children()))[:27]
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 512, kernel_size=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ELU(),
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 256, kernel_size=3),
            torch.nn.BatchNorm2d(256),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(kernel_size=3)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 128, kernel_size=3),
            torch.nn.BatchNorm2d(128),
            torch.nn.ELU()
        )

        # Don't update params in VGG16
        for param in vgg16_bn.parameters():
            param.requires_grad = False

    def forward(self, rendering_images):
        # print(rendering_images.size())  # torch.Size([batch_size, n_views, img_c, img_h, img_w])
        rendering_images = rendering_images.permute(1, 0, 2, 3, 4).contiguous()
        rendering_images = torch.split(rendering_images, 1, dim=0)
        image_features = []

        for img in rendering_images:
# torch.Size([1, 512, 28, 28])
# torch.Size([1, 512, 28, 28])
# torch.Size([1, 256, 8, 8])
# torch.Size([1, 128, 6, 6])
# torch.Size([1, 1, 128, 6, 6])

# torch.Size([1, 512, 28, 28])
# torch.Size([1, 512, 26, 26])
# torch.Size([1, 256, 8, 8])
# torch.Size([1, 128, 8, 8])
# torch.Size([1, 1, 128, 8, 8])
            features = self.vgg(img.squeeze(dim=0))
            print(features.size())    # torch.Size([batch_size, 512, 28, 28])
            features = self.layer1(features)
            print(features.size())    # torch.Size([batch_size, 512, 26, 26])
            features = self.layer2(features)
            print(features.size())    # torch.Size([batch_size, 512, 24, 24])
            features = self.layer3(features)
            print(features.size())    # torch.Size([batch_size, 256, 8, 8])
            image_features.append(features)

        image_features = torch.stack(image_features).permute(1, 0, 2, 3, 4).contiguous()
        print(image_features.size())  # torch.Size([batch_size, n_views, 256, 8, 8])
        return image_features

In [119]:
rand=torch.rand(1,1,256).rehsape(2048,2,2,2)

AttributeError: 'Tensor' object has no attribute 'rehsape'

In [153]:
class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        

        # Layer Definition
        self.layer1 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(256, 128, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(128),
            torch.nn.ReLU()
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(64),
            torch.nn.ReLU()
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(64, 32, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(32),
            torch.nn.ReLU()
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(32, 8, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(8),
            torch.nn.ReLU()
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(8, 1, kernel_size=1, bias=False),
            torch.nn.Sigmoid()
        )

    def forward(self, image_features):
        # [1, 1, 128, 6, 6]
        image_features = image_features.permute(1, 0, 2, 3, 4).contiguous()
        image_features = torch.split(image_features, 1, dim=0)
        gen_volumes = []
        raw_features = []

        for features in image_features:

#             gen_volume = features.view(-1, 256, 2, 2, 2)
            gen_volume = features.view(-1, 128, 6, 6, 1)
            # print(gen_volume.size())   # torch.Size([batch_size, 156, 2, 2, 2])
#             gen_volume = self.layer1(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 128, 4, 4, 4])
            gen_volume = self.layer2(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 64, 8, 8, 8])
            gen_volume = self.layer3(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 32, 16, 16, 16])
            gen_volume = self.layer4(gen_volume)
            raw_feature = gen_volume
            # print(gen_volume.size())   # torch.Size([batch_size, 8, 32, 32, 32])
            gen_volume = self.layer5(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 1, 32, 32, 32])
            raw_feature = torch.cat((raw_feature, gen_volume), dim=1)
            # print(raw_feature.size())  # torch.Size([batch_size, 9, 32, 32, 32])

            gen_volumes.append(torch.squeeze(gen_volume, dim=1))
            raw_features.append(raw_feature)

        gen_volumes = torch.stack(gen_volumes).permute(1, 0, 2, 3, 4).contiguous()
        raw_features = torch.stack(raw_features).permute(1, 0, 2, 3, 4, 5).contiguous()
        # print(gen_volumes.size())      # torch.Size([batch_size, n_views, 32, 32, 32])
        # print(raw_features.size())     # torch.Size([batch_size, n_views, 9, 32, 32, 32])
        return raw_features, gen_volumes

In [136]:
rand=torch.rand([1, 1, 128, 6, 6])
image_features = out1.permute(1, 0, 2, 3, 4).contiguous()
image_features = torch.split(image_features, 1, dim=0)

In [143]:
type(image_features[0])

torch.Tensor

In [190]:
encoder=Encoder()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [191]:
decoder=Decoder()

In [54]:
import torch.nn as nn

In [55]:
class model(nn.Module):
    def __init__(self,encoder,decoder):
        super(model,self).__init__()
        self.encoder=encoder
        self.decoder=decoder
    def forward(self,x):
        x=self.encoder(x)
        # x = x.view(-1, 256, 2, 2, 2)
        x=self.decoder(x)
        return x

In [192]:
checkpoint = torch.load("/kaggle/input/qqqqqq/Pix2Vox-F-ShapeNet.pth")

/tmp/ipykernel_36/4220080922.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/input/qqqqqq/Pix2Vox-F-ShapeNet.pth")


In [193]:
epoch_idx = checkpoint['epoch_idx']

In [194]:
epoch_idx

245

In [195]:
checkpoint['encoder_state_dict']

OrderedDict([('module.vgg.0.weight',
              tensor([[[[-1.3644e-01,  3.4100e-01, -4.8989e-01],
                        [-7.6665e-02, -1.4044e-01,  1.4004e-01],
                        [-4.0409e-01, -9.2861e-02,  7.8583e-02]],
              
                       [[-7.9438e-03,  4.6198e-02,  3.2642e-02],
                        [-4.0754e-01, -4.5644e-02,  3.5134e-01],
                        [-3.5688e-01,  2.6731e-02,  4.0426e-01]],
              
                       [[ 4.8771e-01,  2.1508e-01,  9.1495e-02],
                        [ 2.8036e-01,  1.2694e-05, -2.4137e-01],
                        [-1.9255e-02, -2.5905e-01, -9.8193e-02]]],
              
              
                      [[[-6.9555e-02,  2.7304e-01,  4.1057e-01],
                        [-7.8063e-02,  2.1053e-01,  2.4030e-01],
                        [ 1.0975e-01, -2.7798e-01, -2.7805e-01]],
              
                       [[-7.1334e-01, -2.3772e-01, -3.9419e-01],
                        [-1.9547e-01, 

In [196]:
if torch.cuda.is_available():
    encoder = torch.nn.DataParallel(encoder).cuda()
    decoder = torch.nn.DataParallel(decoder).cuda()

In [197]:
encoder.load_state_dict(checkpoint['encoder_state_dict'])

<All keys matched successfully>

In [198]:
decoder.load_state_dict(checkpoint['decoder_state_dict'])

<All keys matched successfully>

In [78]:
Model = model(encoder,decoder)

In [199]:
img=cv.imread("/kaggle/input/bbbbbbbb/0093.png")
img=cv.resize(img,(224,224))
img=torch.from_numpy(img).float().view(-1,224,224)

In [200]:
img.shape

torch.Size([3, 224, 224])

In [201]:
img=img.unsqueeze(0).unsqueeze(0)

In [202]:
img.shape

torch.Size([1, 1, 3, 224, 224])

In [203]:
out1=encoder(img)

torch.Size([1, 512, 28, 28])
torch.Size([1, 512, 28, 28])
torch.Size([1, 256, 8, 8])
torch.Size([1, 128, 6, 6])
torch.Size([1, 1, 128, 6, 6])


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [179]:
out1.shape

torch.Size([1, 1, 128, 6, 6])

In [204]:
out=decoder(out1)

In [205]:
output=out[0]

In [184]:
output.shape

torch.Size([1, 1, 9, 48, 48, 8])

In [186]:
!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.1/700.1 kB 4.1 MB/s eta 0:00:00a 0:00:01m


In [206]:
import torch
import numpy as np
from skimage import measure
import trimesh

def tensor_to_obj(tensor, threshold=0.5, filename='model.obj'):
    # Ensure tensor is on CPU and convert to numpy array
    voxels = tensor.cpu().detach().numpy()
    
    # Remove batch and channel dimensions if present
    if voxels.shape[0] == 1:
        voxels = voxels[0]
    if voxels.shape[0] == 1:
        voxels = voxels[0]
    
    # Combine the last dimension (8) into the others to make it 3D
    voxels = voxels.reshape(9, 48, 48, 8).transpose(1, 2, 3, 0).reshape(48, 48, 72)
    
    # Use marching cubes to get the mesh
    vertices, faces, _, _ = measure.marching_cubes(voxels, threshold)

    # Create a trimesh object
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    
    # Save as OBJ
    mesh.export(filename)
    
    print(f"Model saved as {filename}")

# Example usage
# tensor = torch.randn(1, 1, 9, 48, 48, 8)  # Replace this with your actual tensor
tensor_to_obj(output, filename='3d_mnist_model.obj')

Model saved as 3d_mnist_model.obj


In [ ]:









# print(Model)


# img=img.convert_to_tensor()



# x=torch.randn(1,1,3, 224, 224)
# print(x.shape)
# print(Model(x).shape)


In [10]:
import cv2 as cv

In [36]:
img=cv.imread("/kaggle/input/bbbbbbbb/0093.png")

In [37]:
type(img)

numpy.ndarray

In [38]:
img.shape

(500, 379, 3)

In [39]:
img=cv.resize(img,(224,224))

In [40]:
type(img)

numpy.ndarray

In [41]:
img=img.view(-1,224,224)

TypeError: view() takes from 0 to 2 positional arguments but 3 were given

In [33]:
img=img.float()

In [35]:
type(img)

torch.Tensor

In [44]:

# img=img.convert_to_tensor()
# img=torch.from_numpy(img).float()

img=img.unsqueeze(0).unsqueeze(0)

# x=torch.randn(1,1,3, 224, 224)
# print(x.shape)
# print(Model(x).shape)
print(encoder(img).shape)

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [8]:
x=torch.rand(16,1,3,224,224)
encoder(x)

tensor([[[[[ 2.8042e+00,  4.6067e-01,  9.3434e-01,  ..., -2.2924e-01,
            -1.6860e-01,  7.5713e-01],
           [ 7.0829e-02,  7.3486e-01, -5.8765e-01,  ...,  1.1128e+00,
            -3.1826e-01,  8.3921e-01],
           [-3.8712e-01,  1.0639e+00, -7.0990e-01,  ...,  2.3116e-01,
            -1.3939e-01,  1.1504e+00],
           ...,
           [ 2.3133e-01, -7.6070e-01, -2.4583e-01,  ...,  7.6493e-01,
            -5.5759e-01,  2.6145e+00],
           [ 1.1542e+00, -5.0894e-01, -4.7033e-01,  ..., -6.3919e-01,
             6.0548e-01,  1.2903e+00],
           [ 2.3429e-01,  4.9251e-01,  1.6428e+00,  ..., -8.6822e-01,
            -3.2450e-01, -5.0321e-01]],

          [[-8.4073e-01,  4.3254e-02, -4.1820e-01,  ...,  1.2874e+00,
             7.9539e-02,  2.5669e+00],
           [ 1.0433e+00, -2.6519e-01,  9.6795e-01,  ...,  9.0870e-01,
            -2.8022e-01, -3.8838e-01],
           [ 7.9655e-01, -7.2384e-01, -2.8887e-03,  ..., -7.4136e-01,
             3.7464e-01, -4.2114e-01],
 